# Imports

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace
import pyarrow as pa
import pyarrow.parquet as pq

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::218606466161:role/LabRole
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 11829a74-a5dd-4bdb-b92c-e189cd60f626
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session 11829a74-a5dd-4bdb-b92c-e189cd60f626 to get into ready status...
Session 11829a74-a5dd-4bdb-b92c-e189cd60f626 has been created.



In [2]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("ReadS3DataEmpregados") \
    .getOrCreate()

# Construa o caminho completo para o arquivo no S3
s3_path = f"s3://218606466161-raw/Empregados/*.csv"
output_s3_path = "s3://218606466161-trusted/Empregados/"

# Leia o arquivo CSV do S3
df_empregados = spark.read.option("encoding", "UTF-8").csv(s3_path, header=True, inferSchema=True, sep='|')


In [3]:
#df_empregados.show()

+--------------------+-------------+-------------+--------------+--------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+-----+-----------------+----------------------+-----------------+--------------+------------------------+-------------------------+---------------------------------+----------------------------------+--------+--------------------+-------------+
|       employer_name|reviews_count|culture_count|salaries_count|benefits_count|    employer-website|employer-headquarters|employer-founded|   employer-industry|    employer-revenue|                 url|Geral|Cultura e valores|Diversidade e inclusão|Qualidade de vida|Alta liderança|Remuneração e benefícios|Oportunidades de carreira|Recomendam para outras pessoas(%)|Perspectiva positiva da empresa(%)|Segmento|                Nome|match_percent|
+--------------------+-------------+-------------+--------------+--------------+--------------------+---

In [4]:
#Agrupando e tirando a media
df_empregados = df_empregados.groupBy("Nome").agg({
        'Perspectiva positiva da empresa(%)': 'mean',
        'Remuneração e benefícios': 'mean'
    })
#Renomeando colunas
df_empregados = df_empregados.withColumnRenamed("avg(Perspectiva positiva da empresa(%))", "satisfacao_empregado") \
                          .withColumnRenamed("avg(Remuneração e benefícios)", "satisfacao_salario")

In [5]:
df_empregados.write.format("parquet").mode("overwrite").option("path", f"{output_s3_path}").save()